In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [5]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_33364/2634772801.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)


In [6]:
# prover calculates settings, send to verifier
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [10], "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====
scale:  [10]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":10,"param_scale":10,"scale_rebase_multiplier":10,"lookup_range":[-16,180224],"logrows":18,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":906,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,10],"model_input_scales":[10],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":1024.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_33364/788883855.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<torch.log(torch.tensor(1.01)), self.w)


In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 19.89814805984497 seconds
Theory output:  tensor(47.6981)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.6982421875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[14672592657000597156, 2402501382455789823, 5345685197495592745, 3011024057811024003], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12156132031024860846, 9861177863383639274, 10629998272028772020, 2786626043251147566]]], 'proof': '19c1cb69c76c75a516ffef76116a6ea52872328ecb306ce564276d4c00de0f971a50d78fd1f85124a563e33261de4822806cb2820663908e751ce1d970157b9a11a9d8ec1b1a64432bb48e7b6721be1bd115707f820ec6da20bc6206df4263210782c6ef7dca6a7099fe54575cb56f33c59c9babb1c932a3b640db8a936bdd261fecf52050e84bf3979b44ee2ba91b31f777bc922a675bb67c8ec6139ebe19ce188f0d7b5986058c00ea1cebebb9a408b7cf7dba03cab61850383ce6065dac9b0af2f4ceab12ef737e5ac5ba418a2cbebf72e6e36623b066ae838f141701432f0fb495e1df18e667545819f1758940b234f8306cde1dec07612ad07d5649e86706b435abdbfd926d90400bf1b78444387df1aa4d46b2411367645759a2aa8d092e15db144328ba1405244857f35975abdc138eb4c9aea162ac2c1ac7b0ca19ee160599e338c2b38c415779a2ee0d7f7d811a88eb6de4f2e0771a5001731f97

In [8]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[14672592657000597156, 2402501382455789823, 5345685197495592745, 3011024057811024003], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12156132031024860846, 9861177863383639274, 10629998272028772020, 2786626043251147566]]]
proof boolean:  1.0
proof result 1 : 47.6982421875
verified
